In [2]:
import cvxpy as cp
import numpy as np

# Problem setup (normalized units)
M = 10        # Number of time slots
K = 4         # Number of cells (users)
tau = 1.0     # Slot duration (normalized to 1)
B = 1.0       # Bandwidth (normalized to 1 Hz)
SNR_dB = np.array([10, 12, 11, 9])  # SNRs per user
SNR = 10 ** (SNR_dB / 10)           # Convert to linear scale

# Normalized rate per beam (in bits per time slot)
R_k = B * np.log2(1 + SNR)  # Unit: bits/second in normalized terms

# Set normalized demands in bits (low values now make sense)
d_k = np.array([1.0, 2.0, 3.0, 4.0])  # Reasonable given normalized R_k

# Binary decision variable: x[i, k] = 1 if cell k is served at time i
x = cp.Variable((M, K), boolean=True)

# Constraints
constraints = []

# (C1) Meet demand for each user
for k in range(K):
    constraints.append(cp.sum(x[:, k]) * R_k[k] * tau >= d_k[k])

# (C2) Only one beam active per time slot
for i in range(M):
    constraints.append(cp.sum(x[i, :]) <= 1)

# Objective: feasibility only
objective = cp.Minimize(0)
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve(solver=cp.ECOS_BB)

# Output schedule
if x.value is not None:
    schedule = np.round(x.value).astype(int)
    print("Beam schedule (time slots × cells):\n", schedule)
else:
    print("Problem is infeasible")


Beam schedule (time slots × cells):
 [[0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]]
